In [1]:
import pandas as pd
import numpy as np
import datetime
import time
import utm
from pyproj import Proj

pd.options.display.max_columns = None

In [2]:
df = pd.read_excel("2021_Accidentalidad.xlsx")
df.head(2)

FileNotFoundError: [Errno 2] No such file or directory: '2021_Accidentalidad.xlsx'

# Duplicados

Eliminamos duplicados

In [ ]:
df.drop_duplicates(inplace = True)

# Tipado de columnas

In [ ]:
df.dtypes

Cambiamos el tipado de columnas que queremos

In [ ]:
df['cod_distrito'] = pd.to_numeric(df["cod_distrito"], errors='coerce').astype('Int64')
df['cod_lesividad'] = pd.to_numeric(df["cod_lesividad"], errors='coerce').astype('Int64')
df["positiva_droga"] = df["positiva_droga"].astype("category")

In [ ]:
df.shape

# Añadimos columnas

Añadimos columna de interés como el día de la semana

In [ ]:
df['dia_semana'] = df['fecha'].dt.day_name()

In [ ]:
df.head(1)

# Eliminamos columnas

Eliminamos la columna de "positiva_droga" por tener la mayoría de campos en nulo y no aportar gran valor

In [ ]:
df_limpio = df.drop(["positiva_droga"], axis = 1)

In [ ]:
df_limpio.head(1)

# Gestión de Nulos

Eliminamos o sustituimos Nulos

In [ ]:
df_limpio.isnull().sum()

1️⃣ Columna "positiva_alcohol"

In [ ]:
df_limpio["positiva_alcohol"].unique()

In [ ]:
df_limpio["positiva_alcohol"].value_counts()

Sustituimos los nulos de la columna "positiva_alcohol" por "Sin especificar"

In [ ]:
df_limpio['positiva_alcohol'] = df['positiva_alcohol'].fillna("Sin especificar")

2️⃣ Columna "tipo_accidente"

In [ ]:
df_limpio["tipo_accidente"].unique()

Sustituimos los nulos de la columna "tipo_accidente" por "Otro"

In [ ]:
df_limpio['tipo_accidente'] = df['tipo_accidente'].fillna("Otro")

3️⃣ Columna "Tipo_vehiculo"

In [ ]:
df_limpio["tipo_vehiculo"].unique()

In [ ]:
df_limpio["tipo_vehiculo"].value_counts() / df_limpio.shape[0] * 100

Todos los tipos de vehículo con una representación muy baja los agrupamos en "Otros"

In [ ]:
df_limpio ['tipo_vehiculo'] = df_limpio ['tipo_vehiculo']. replace ([
       'Sin especificar', 'Maquinaria de obras',
       'Tractocamión', 'Otros vehículos con motor',
       'Cuadriciclo no ligero', 'Vehículo articulado', 'Autocaravana',
       'Bicicleta EPAC (pedaleo asistido)', 'Autobús articulado',
       'Autobus EMT', 'Ambulancia SAMUR', 'Cuadriciclo ligero',
       'Patinete', 'Ciclomotor de tres ruedas', 'Semiremolque',
       'Moto de tres ruedas > 125cc', 'Ciclo', 'Maquinaria agrícola',
       'Remolque', 'Moto de tres ruedas hasta 125cc',
       'Ciclomotor de dos ruedas L1e-B', 'Tren/metro',
       'Otros vehículos sin motor', 'Camión de bomberos',
       'Ciclo de motor L1e-A', 'Tranvía', 'Autobús articulado EMT', np.nan], ["Otros","Otros","Otros","Otros","Otros","Otros","Otros","Otros","Otros","Otros","Otros","Otros","Otros","Otros","Otros","Otros","Otros","Otros","Otros","Otros","Otros","Otros","Otros","Otros","Otros","Otros","Otros", "Otros"])

4️⃣ Distrito

In [ ]:
df_limpio["distrito"].unique()

In [ ]:
df_limpio["distrito"].isnull().sum()

In [ ]:
df_limpio[df_limpio["distrito"].isnull()]

Sustituimos los nulos de distrito

In [ ]:
df_limpio["distrito"].replace(np.nan, "Arganzuela", inplace = True)

In [ ]:
df_limpio["cod_distrito"].replace(np.nan, "2", inplace = True)

5️⃣ Columna "Tipo_persona"

In [ ]:
df_limpio["tipo_persona"].unique()

In [ ]:
df_limpio[df_limpio["tipo_persona"].isnull()]

Manipulamos la columna "Tipo_persona" para que es los nulos diga "sin especificar"

In [ ]:
df_limpio["tipo_persona"].replace(np.nan, "Sin especificar", inplace = True)

In [ ]:
#para ver culquier nulo de cualquier columna
#nan_rows = df_limpio[df_limpio.isnull().any(1)]
#nan_rows.head(10)

6️⃣ Columna "lesividad"

In [ ]:
df_limpio["lesividad"].unique()

In [ ]:
df_limpio["lesividad"].value_counts()

In [ ]:
df_limpio['lesividad'] = df['lesividad'].fillna("Sin especificar")

 7️⃣ Columna "Coordinadas"

In [ ]:
df_limpio[df_limpio["coordenada_x_utm"].isnull()]

8️⃣ Columna "estado_meteorológico"

In [ ]:
df_limpio["estado_meteorológico"].unique()

In [ ]:
df_limpio["estado_meteorológico"].value_counts()

In [ ]:
df_limpio['estado_meteorológico'] = df['estado_meteorológico'].fillna("Se desconoce")

# Manipulación de columnas

1️⃣ Distrito --> Poner el string en minúsculas con la primera letra mayúscula

In [ ]:
df_limpio["distrito"] = df_limpio["distrito"].str.capitalize()
df_limpio.head(2)

2️⃣ Localización --> Poner el string en minúsculas con la primera letra mayúscula

In [ ]:
df_limpio["localizacion"] = df_limpio["localizacion"].str.capitalize()
df_limpio.head(2)

# Eliminación de columnas

In [ ]:
eliminar = ["cod_lesividad"]

In [ ]:
df_limpio = df_limpio.drop(eliminar, axis = 1)

# Groupby

Agrupación por número de expediente

In [ ]:
df_pie = pd.DataFrame(df_limpio.groupby(['num_expediente', 'distrito'])["tipo_vehiculo", "tipo_accidente", "tipo_persona"].count())

df_pie.head(1)

In [ ]:
df_pie.reset_index(inplace = True)
df_pie.head(2)

In [ ]:
df_pie.shape

Agrupación por distrito

In [ ]:
df_pie_1 = pd.DataFrame(df_pie.groupby(['distrito'])["num_expediente"].count())

df_pie_1.head(2)

Agrupación por dia de la semana

In [ ]:
df_pie_3 = pd.DataFrame(df_limpio.groupby(['num_expediente', 'dia_semana'])["tipo_vehiculo", "tipo_accidente", "tipo_persona"].count())

df_pie_3.head(1)

In [ ]:
df_pie_3.reset_index(inplace = True)
df_pie_3.head(2)

In [ ]:
df_pie_3.shape

In [ ]:
df_pie_4 = pd.DataFrame(df_pie_3.groupby(['dia_semana'])["num_expediente"].count())

df_pie_4.head(2)

In [ ]:
#df_pie2 = pd.DataFrame(df_pie1.groupby(["distrito", "num_expediente"])["num_expediente"].count())

#df_pie2.head(20)

# Coordenadas

In [ ]:
test_coordenada = utm.to_latlon(437390.155, 4476258.031, 30, 'T')
test_coordenada

In [ ]:
latitud =test_coordenada[0]

In [ ]:
longitud = test_coordenada[1]

In [ ]:
def utm_lat(lat, lon):
    try:
        coordenada = utm.to_latlon(lat, lon, 30, 'T')
        return coordenada[0]
    except:
        return None

In [ ]:
def utm_lon(lat, lon):
    try:
        coordenada = utm.to_latlon(lat, lon, 30, 'T')
        return coordenada[1]
    except:
        return None

In [ ]:
df_limpio["latitud"] = df_limpio.apply(lambda datos: utm_lat
                                           (datos["coordenada_x_utm"],
                                            datos["coordenada_y_utm"]), axis =1)

In [ ]:
df_limpio["longitud"] = df_limpio.apply(lambda datos: utm_lon
                                           (datos["coordenada_x_utm"],
                                            datos["coordenada_y_utm"]), axis =1)

In [ ]:
df_limpio.head()

In [ ]:
df_limpio["fecha"].unique()

# Guardamos CSV

In [ ]:
df_limpio.to_excel("df_limpio_transformado_1.xlsx")